# Snorkel Model v3

In [1]:
import snorkel
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from snorkel.labeling import LabelingFunction, PandasLFApplier
from snorkel.labeling.model.label_model import LabelModel

In [2]:
# Load Data
Labels = pd.read_csv('Labels_renamed.csv')
Labels.columns = ['Labels']
abstract_df = pd.read_csv('preprocessed_data_Jul14.csv')
abstract_df = abstract_df['concatenated_title_abstract'].to_frame(name='concatenated_title_abstract')

# Define Labeling Functions
LABEL_DICT = {label: i for i, label in enumerate(Labels['Labels'].tolist())}

In [4]:
# Define ABSTAIN constant
ABSTAIN = -1

def lf_observatories(row):
    observatories = pd.read_csv('Astronomical Observatories.csv')['Name'].tolist()
    abstract = row['concatenated_title_abstract']
    for observatory in observatories:
        if observatory in abstract:
            return LABEL_DICT.get(observatory, ABSTAIN)
    return ABSTAIN

def lf_radio_telescopes(row):
    telescope_names = pd.read_csv('List of radio telescopes.csv')['Name'].tolist()
    abstract = row['concatenated_title_abstract']
    for telescope_name in telescope_names:
        if telescope_name in abstract:
            return LABEL_DICT.get(telescope_name, ABSTAIN)
    return ABSTAIN

def lf_solar_optical_telescopes(row):
    telescope_names = pd.read_csv('List of Solar Telescope Optical.csv')['Name/Observatory'].tolist()
    abstract = row['concatenated_title_abstract']
    for telescope_name in telescope_names:
        if telescope_name in abstract:
            return LABEL_DICT.get(telescope_name, ABSTAIN)
    return ABSTAIN

def lf_solar_radio_telescopes(row):
    telescope_names = pd.read_csv('List of Solar Telescope Radio.csv')['Name/Observatory'].tolist()
    abstract = row['concatenated_title_abstract']
    for telescope_name in telescope_names:
        if telescope_name in abstract:
            return LABEL_DICT.get(telescope_name, ABSTAIN)
    return ABSTAIN

def lf_space_telescopes(row):
    telescope_names = pd.read_csv('List of space telescopes.csv')['Name'].tolist()
    abstract = row['concatenated_title_abstract']
    for telescope_name in telescope_names:
        if telescope_name in abstract:
            return LABEL_DICT.get(telescope_name, ABSTAIN)
    return ABSTAIN

def lf_telescope_types(row):
    telescope_types = ['radio telescope', 'solar telescope', 'space telescope']
    abstract = row['concatenated_title_abstract']
    for telescope_type in telescope_types:
        if telescope_type in abstract:
            return LABEL_DICT.get(telescope_type, ABSTAIN)
    return ABSTAIN


# Wrap your functions using Snorkel's LabelingFunction
lfs = [
    LabelingFunction(name="LF_Observatories", f=lf_observatories),
    LabelingFunction(name="LF_RadioTelescopes", f=lf_radio_telescopes),
    LabelingFunction(name="LF_SolarOpticalTelescopes", f=lf_solar_optical_telescopes),
    LabelingFunction(name="LF_SolarRadioTelescopes", f=lf_solar_radio_telescopes),
    LabelingFunction(name="LF_SpaceTelescopes", f=lf_space_telescopes),
    LabelingFunction(name="LF_TelescopeTypes", f=lf_telescope_types)
]

In [5]:
# Apply the labeling functions to the abstract data
applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(abstract_df)

100%|██████████| 63999/63999 [10:18<00:00, 103.54it/s]


In [6]:
# Train a Snorkel label model
NUM_LABELS = len(Labels)
label_model = LabelModel(cardinality=NUM_LABELS)
label_model.fit(L_train, n_epochs=100, log_freq=10, seed=123)

INFO:root:Computing O...
INFO:root:Estimating \mu...
100%|██████████| 100/100 [13:54<00:00,  8.34s/epoch]
INFO:root:Finished Training


In [7]:
# Add the 'label' column to the DataFrame
abstract_df['label'] = label_model.predict(L_train)

In [8]:
# Map the labels to their respective names
abstract_df['label_name'] = abstract_df['label'].apply(lambda x: Labels.iloc[x]['Labels'] if x != ABSTAIN else 'ABSTAIN')
abstract_df

,concatenated_title_abstract,label,label_name
0,KINEMATIC TREATMENT OF CORONAL MASS EJECTION E...,-1,ABSTAIN
1,The imaging performance of the Hubble Space Te...,990,Hubble Space Telescope
2,Numerical Simulations of Mass Outflows Driven ...,-1,ABSTAIN
3,The Origin of X-shaped Radio Galaxies: Clues f...,-1,ABSTAIN
4,The Ghost of Sagittarius and Lumps in the Halo...,-1,ABSTAIN
...,...,...,...
63994,Accretion Disk Spectra of the Ultra-luminous X...,-1,ABSTAIN
63995,The ghost of a dwarf galaxy: fossils of the hi...,-1,ABSTAIN
63996,Observations of the Solar Corona from Space Sp...,-1,ABSTAIN
63997,Constraints on the topology of the Universe de...,1016,WMAP


In [17]:
print(abstract_df.iloc[203])

concatenated_title_abstract    LBDS 53W091: An Old, Red Galaxy at z = 1.552* ...
label                                                                         -1
label_name                                                               ABSTAIN
Name: 203, dtype: object


In [9]:
# save to CSV
abstract_df.to_csv('labeled_data_snorkel_v3.csv', index=False)

In [10]:
label_counts = abstract_df['label_name'].value_counts()
print(label_counts)

label_name
ABSTAIN                                               51247
Hubble Space Telescope                                 3036
Astron                                                  937
IRAS                                                    805
XMM-Newton                                              750
                                                      ...  
Paranal Observatory                                       1
Ankara University Observatory                             1
Neutron Star Interior Composition Explorer (NICER)        1
Ege University Observatory                                1
Very Small Array (VSA)                                    1
Name: count, Length: 197, dtype: int64


In [11]:
label_counts_df = label_counts.reset_index()
label_counts_df.columns = ['Label', 'Count']

# save to CSV
label_counts_df.to_csv('label_counts_Snorkel_v3.csv', index=False)
label_counts_df

,Label,Count
0,ABSTAIN,51247
1,Hubble Space Telescope,3036
2,Astron,937
3,IRAS,805
4,XMM-Newton,750
...,...,...
192,Paranal Observatory,1
193,Ankara University Observatory,1
194,Neutron Star Interior Composition Explorer (NI...,1
195,Ege University Observatory,1
